In [ ]:
# zipファイルの解凍
# このコードを実行すると、アップロードしたzipファイルが解凍されます。
!unzip "ex2_data.zip"

Archive:  ex2_data.zip
replace index_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# ライブラリのインポート
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn import datasets, svm, metrics
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
# テキストP177を参考に画像ファイルを1次元のndarrayに変換する関数を作成する

# * 引数 filename ... 画像ファイルのパスを受け取る
# * 戻り値 ... 画像を1次元配列にした結果を返す
def read_dir(filename):
    # 自分で用意した手書きの画像ファイルを読み込む
    my_img = cv2.imread(filename)

    # 画像をグレースケールにする
    my_img = cv2.cvtColor(my_img, cv2.COLOR_BGR2GRAY)
    
    # 画像を32×32にリサイズする
    my_img = cv2.resize(my_img, (32, 32))
    
    # 二次元を一次元に変換する(データ数は32×32個となる)
    # my_img = my_img.reshape((-1,))
    my_img = my_img.reshape(-1, 32*32)

    return my_img

In [ ]:
# ファイルの一覧(index_train.csv)を読み込み、データフレームを作成する
df_index = pd.read_csv('index_train.csv' ,encoding='shift-jis')
print(df_index.head())

                  filename target
0    ./traindata/a (1).png      ア
1   ./traindata/a (10).png      ア
2  ./traindata/a (100).png      ア
3   ./traindata/a (11).png      ア
4   ./traindata/a (12).png      ア


In [ ]:
# 読み込んだデータフレームのファイル名で画像ファイルを読み込み、特徴量Xに代入する
# 空のデータフレームを作成しておく
X = pd.DataFrame()

# forで1レコードずつ取り出す
# イテレータ：繰り返す
for index, row in df_index.iterrows():
    # 作成したread_dir関数を使い、画像を一次元配列に変換し、特徴量とする
    # 引数→対象の画像ファイルのフルパス
    ret = read_dir(row['filename'])

    # 変換後の配列(ndarray型)からDataFrameを生成し、特徴量Xに追加する
    # X = X.append(pd.Series(ret), ignore_index=True)
    X = X.append(pd.DataFrame(ret))

# 読み込んだデータフレームのtarget列を、正解データyに代入する
y = df_index['target']

In [ ]:
# 特徴量X、正解データyをデータを学習用とテスト用に分割する
# テスト用データは全体の20%とする
# 乱数シードは0とする
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# SVCのモデルを生成する
# 乱数シードは0とする
clf = svm.SVC(random_state=0)

# 学習用データを学習する
clf.fit(X_train, y_train)

# テスト用データで予測して精度を確認する
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9


In [ ]:
# ⑬学習済モデルを「ex2_model.pkl」という名前で保存する
with open("ex2_model.pkl", "wb") as fp:
    pickle.dump(clf, fp)